In [ ]:
import os.path as op
import mne
from mne.preprocessing import ICA
from mne.event import define_target_events
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import os
import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)

In [18]:

T2M_sub_ids = [
    'prex006','prex009','prex010','prex011','prex012',
    'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
    'prex026','prex027','prex028'
    ,'prex033','prex037','prex038','prex040',
    'prex041','prex042','prex044','prex045',
    'prex046','prex047','prex048','prex049','prex050','prex052',
    'prex034','prex043','prex035'
]
#去掉了 5和9被试
T1_sub_ids = [
    'prex006M','prex009M','prex010M','prex011M','prex012M',
    'prex015M','prex016M','prex017M','prex018M','prex019M','prex022M','prex023M',
    'prex026M','prex027M','prex028M'
    ,'prex033M','prex037M','prex038M','prex040S'
    ,'prex041M','prex042M','prex044M','prex045M'
    ,'prex046M','prex047M','prex048M','prex049S','prex050M','prex052M',
    'prex034M','prex043M','prex035M'
]

T2S_sub_ids = [
    'prex006S','prex009S','prex010S','prex011S','prex012S',
    'prex015S','prex016S','prex017S','prex018S','prex019S','prex022S','prex023S',
    'prex026S','prex027S','prex028S'
    ,'prex033S','prex037S','prex038S','prex040M'
    ,'prex041S','prex042S','prex044S','prex045S'
    ,'prex046S','prex047S','prex048S','prex049M','prex050S','prex052S',
    'prex034S','prex043S','prex035S'
]
data_path = 'E:\Python LYW\RSA\spaced_learning\pre10\\analysis\data\\7evoked\\'

In [19]:

list_evoked_T1S = list()
list_evoked_T2S = list()
list_evoked_T2M = list()

for sub_id in T1_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T1S.append(evoked)

for sub_id in T2S_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T2S.append(evoked)

for sub_id in T2M_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T2M.append(evoked)

Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked\prex006M-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (102)
        0 CTF compensation matrices available
        nave = 47 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.2, 0] sec)
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked\prex009M-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (102)
        0 CTF compensation matrices available
        nave = 44 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked\prex010M-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (102)
        0 CTF compensation matrices available
        nave

In [20]:
#先考虑单个被试，单个channel的情况
#选出对应的名字的channel

# # 知道list的值，获取list的编号，用这个编号获取对应的data
# evoked_01 = list_evoked_T1S[0]
# ch_names = evoked_01.ch_names
# print(ch_names.index('Cz'))
# index = ch_names.index('Cz')
# ch_data = evoked_01.data[index]
#
# # 怎么截取，就变成切片问题。需要考虑resolution
# #  N400 取300到450之间
# # （300+200）/2 = 250
# #  (450+200)/2 = 325
# ch_data_part = ch_data[250:325]

In [21]:
# 在上面基础上加入被试循环
# 先对一个list下手 list_evoked_T1S

# 想写成一个函数了。
# list_ch_data_part  = []
# for evoked_T1S in list_evoked_T1S:
#     ch_names = evoked_T1S.ch_names
#     print(ch_names.index('Cz'))
#     index = ch_names.index('Cz')
#     ch_data = evoked_T1S.data[index]
#     ch_data_part = ch_data[250:325]
#     list_ch_data_part.append(ch_data_part)

In [22]:
# 区域eeg赋值有了，考虑peak或者average，得到统计量的list
# #
# list_ch_mean_part = []
# for ch_data_part in list_ch_data_part:
#     ch_mean_part = ch_data_part.mean()
#     list_ch_mean_part.append(ch_mean_part)

In [35]:
# 区域均值
# 现在把所有条件的list都求出来。
list_ch_data_part_T1S  = []
for evoked in list_evoked_T1S:
    ch_names = evoked.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked.data[index]
    ch_data_part = ch_data[350:400]
    list_ch_data_part_T1S.append(ch_data_part)
list_ch_mean_part_T1S = []
for ch_data_part in list_ch_data_part_T1S:
    #这里是求的均值
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T1S.append(ch_mean_part)

list_ch_data_part_T2S  = []
for evoked in list_evoked_T2S:
    ch_names = evoked.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked.data[index]
    ch_data_part = ch_data[350:400]
    list_ch_data_part_T2S.append(ch_data_part)
list_ch_mean_part_T2S = []
for ch_data_part in list_ch_data_part_T2S:
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T2S.append(ch_mean_part)

list_ch_data_part_T2M  = []
for evoked in list_evoked_T2M:
    ch_names = evoked.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked.data[index]
    ch_data_part = ch_data[350:400]
    list_ch_data_part_T2M.append(ch_data_part)
list_ch_mean_part_T2M = []
for ch_data_part in list_ch_data_part_T2M:
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T2M.append(ch_mean_part)
f = open('./mean_data', 'w')

for item in list_ch_mean_part_T1S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_mean_part_T2S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_mean_part_T2M:
    print(item)
    f.write("%s\n" % item)
f.close()

3.7392084461891413e-06
-6.830361793989728e-07
1.0527910692407465e-06
1.1723218618338443e-07
1.9524207400431453e-06
1.3145252910700782e-06
7.312137021064746e-07
-3.67457909821774e-07
3.2834836836596843e-06
1.7040426973099195e-06
8.515490907955428e-07
4.901086904555456e-06
1.636850155426284e-06
7.018671978465974e-07
6.616240378145203e-08
1.2034379630832766e-06
4.65369086691323e-06
1.0442567955661644e-06
1.6043779399170743e-06
6.110355797619211e-08
1.4241449001321504e-06
1.5193592906991236e-06
2.118760511360753e-06
1.1620090010986634e-06
6.67628814029222e-08
2.161219942908635e-06
9.334248398824752e-07
2.706763498597513e-06
4.4186879990822113e-07
-5.152379895216965e-07
-8.425436124564242e-08
1.1406438713386786e-06
4.140589069574535e-06
-2.6537905619889717e-09
-3.268218140945223e-08
6.436870325352793e-07
1.8918400323817936e-07
1.4457376780109969e-06
3.77751566846047e-07
6.175604781554836e-07
2.5895768216538504e-06
2.025779757960095e-06
1.576041197333196e-06
3.962264826366433e-06
1.149609771

In [36]:
# 写代码直接做ttest不是更方便？
from scipy.stats import ttest_rel
import pandas as pdc
arr_ch_mean_part_T1S = np.array(list_ch_mean_part_T1S)
arr_ch_mean_part_T2M = np.array(list_ch_mean_part_T2M)
arr_ch_mean_part_T2S = np.array(list_ch_mean_part_T2S)

print(ttest_rel((arr_ch_mean_part_T2S-arr_ch_mean_part_T1S),(arr_ch_mean_part_T2M-arr_ch_mean_part_T1S)))



Ttest_relResult(statistic=2.167983013265196, pvalue=0.037955338067114944)


In [12]:
# 做一下峰值的比较

list_ch_data_part_T1S  = []
for evoked_T1S in list_evoked_T1S:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[300:350]
    list_ch_data_part_T1S.append(ch_data_part)
list_ch_peak_part_T1S = []
for ch_data_part in list_ch_data_part_T1S:
    ch_peak_part = min(ch_data_part)
    list_ch_peak_part_T1S.append(ch_peak_part)

list_ch_data_part_T2S  = []
for evoked_T1S in list_evoked_T2S:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[300:350]
    list_ch_data_part_T2S.append(ch_data_part)
list_ch_peak_part_T2S = []
for ch_data_part in list_ch_data_part_T2S:
    ch_peak_part = min(ch_data_part)
    list_ch_peak_part_T2S.append(ch_peak_part)

list_ch_data_part_T2M  = []
for evoked_T1S in list_evoked_T2M:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('CPz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[300:350]
    list_ch_data_part_T2M.append(ch_data_part)
list_ch_peak_part_T2M = []
for ch_data_part in list_ch_data_part_T2M:
    ch_peak_part = min(ch_data_part)
    list_ch_peak_part_T2M.append(ch_peak_part)

f = open('./peak_data', 'w')
for item in list_ch_peak_part_T1S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_peak_part_T2S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_peak_part_T2M:
    print(item)
    f.write("%s\n" % item)
f.close()



1.936829932431484e-06
-2.0661589895571054e-06
1.8944480755532248e-08
-1.1940784539217758e-07
3.6671836461056373e-07
1.0331576203745992e-06
-8.846137593477684e-07
-7.935428361646751e-08
-2.581165556230022e-07
-2.7703771096547442e-06
-8.226777149309341e-07
2.00816578663015e-06
-7.437783662554144e-07
1.0971835926977799e-06
1.721459878539825e-07
-3.329354562582143e-07
1.0742487735119463e-06
-1.0452471698275343e-06
1.2122328779343872e-07
1.2285064441383377e-06
-4.452203550449578e-07
-2.591428203648283e-06
1.6720576490069944e-06
-1.0600843937649392e-06
-3.107287785414703e-07
8.478743451556266e-08
-4.313596442147139e-08
1.1906171671462938e-06
-2.259022619011217e-06
-1.465483477668823e-06
-1.3558128759665468e-06
4.980722184522302e-08
2.930005437524604e-06
-3.2443910214679833e-06
-7.227904569319423e-07
-1.7290701892571502e-07
-4.826037710763376e-07
1.2801803868337403e-06
-1.1565709600738576e-06
-2.62553411904878e-07
7.994574053539599e-07
-1.9424125928930948e-06
-1.1641833398916602e-06
2.1466823

In [9]:
# 潜伏期比较
list_ch_data_part_T1S  = []
for evoked_T1S in list_evoked_T1S:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('Cz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[250:350]
    list_ch_data_part_T1S.append(ch_data_part)
list_ch_peak_part_T1S = []
for ch_data_part in list_ch_data_part_T1S:
    ch_data_part = ch_data_part.tolist()
    ch_peak_part = ch_data_part.index(min(ch_data_part))
    list_ch_peak_part_T1S.append(ch_peak_part)

list_ch_data_part_T2S  = []
for evoked_T1S in list_evoked_T2S:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('Cz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[250:350]
    list_ch_data_part_T2S.append(ch_data_part)
list_ch_peak_part_T2S = []
for ch_data_part in list_ch_data_part_T2S:
    ch_data_part = ch_data_part.tolist()
    ch_peak_part = ch_data_part.index(min(ch_data_part))
    list_ch_peak_part_T2S.append(ch_peak_part)

list_ch_data_part_T2M  = []
for evoked_T1S in list_evoked_T2M:
    ch_names = evoked_T1S.ch_names
    index = ch_names.index('Cz')
    ch_data = evoked_T1S.data[index]
    ch_data_part = ch_data[250:350]
    list_ch_data_part_T2M.append(ch_data_part)
list_ch_peak_part_T2M = []
for ch_data_part in list_ch_data_part_T2M:
    ch_data_part = ch_data_part.tolist()
    ch_peak_part = ch_data_part.index(min(ch_data_part))
    list_ch_peak_part_T2M.append(ch_peak_part)

f = open('./latency_data', 'w')
for item in list_ch_peak_part_T1S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_peak_part_T2S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
for item in list_ch_peak_part_T2M:
    print(item)
    f.write("%s\n" % item)
f.close()

33
17
7
75
99
12
2
40
0
13
68
63
63
57
92
25
33
78
21
0
73
28
18
73
0
66
55
28
5
52
39
18
38
27
19
55
73
31
5
43
82
0
46
56
16
57
75
18
47
89
28
8
67
30
6
89
75
16
45
4
16
45
53
23
39
12
19
85
52
28
51
38
0
1
68
37
47
99
62
23
29
91
24
16
76
35
5
28
70
52
71
24
16
86
46
14


In [10]:
# 想要把多个channel合并成一个再来比较。
# 怎么合并呢，直接按某个维度进行压缩?求平均？

#一个erp放进去，得到对应的channel平均
def get_ROI(list_ROI_ch = ['Cz', 'CPz', 'Pz'], evoked = list_evoked_T1S[0]):
    ch_names = evoked.ch_names
    list_ROI_ch_data_part = []
    for ROI_ch in list_ROI_ch:
        index = ch_names.index(ROI_ch)
        ch_data = evoked.data[index]
        ch_data_part = ch_data[250:350]
        list_ROI_ch_data_part.append(ch_data_part)
    #制作对应的ndarray？还是直接全部相加，这样子倒是可以确定结构
    ROI_ch_data_part = np.zeros(shape=(100,))
    for i in range(len(list_ROI_ch)-1):
        sum_ROI_ch_data_part = ROI_ch_data_part+list_ROI_ch_data_part[i]
    ROI_ch_data_part = sum_ROI_ch_data_part/len(list_ROI_ch)
    return ROI_ch_data_part


In [11]:
#和前面的代码合并，生成多个被试的
list_ch_data_part_T1S  = []
for evoked_T1S in list_evoked_T1S:
    ROI_data = get_ROI(list_ROI_ch = ['CPz'], evoked = evoked_T1S)
    list_ch_data_part_T1S.append(ROI_data)
list_ch_mean_part_T1S = []
for ch_data_part in list_ch_data_part_T1S:
    # ch_data_part = ch_data_part.tolist()
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T1S.append(ch_mean_part)

#写出去
f = open('./mean_data', 'w')

for item in list_ch_mean_part_T1S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")


list_ch_data_part_T2S  = []
for evoked_T2S in list_evoked_T2S:
    ROI_data = get_ROI(list_ROI_ch = ['CPz'], evoked = evoked_T2S)
    list_ch_data_part_T2S.append(ROI_data)
list_ch_mean_part_T2S = []
for ch_data_part in list_ch_data_part_T2S:
    # ch_data_part = ch_data_part.tolist()
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T2S.append(ch_mean_part)



for item in list_ch_mean_part_T2S:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
#和前面的代码合并，生成多个被试的
list_ch_data_part_T2M  = []
for evoked_T2M in list_evoked_T2M:
    ROI_data = get_ROI(list_ROI_ch = ['CPz'], evoked = evoked_T2M)
    list_ch_data_part_T2M.append(ROI_data)
list_ch_mean_part_T2M = []
for ch_data_part in list_ch_data_part_T2M:
    # ch_data_part = ch_data_part.tolist()
    ch_mean_part = ch_data_part.mean()
    list_ch_mean_part_T2M.append(ch_mean_part)


for item in list_ch_mean_part_T2M:
    print(item)
    f.write("%s\n" % item)
f.write("\n")
f.close()

UnboundLocalError: local variable 'sum_ROI_ch_data_part' referenced before assignment